In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
%cd /content/drive/MyDrive/final_project/data
df = pd.read_csv('academy.csv')

/content/drive/MyDrive/final_project/data


# 데이터 전처리

In [ ]:
df['분야명'].unique()

array(['입시.검정 및 보습', '기예(대)', '예능(대)', '국제화', '종합(대)', '기타(대)', '인문사회(대)'],
      dtype=object)

In [ ]:
df.drop(df[df['분야명'] == '독서실'].index, inplace=True)
df.drop(df[df['분야명'] == '직업기술'].index, inplace=True)
df.drop(df[df['분야명'] == '정보'].index, inplace=True)
df.dropna(subset=['분야명'], inplace=True)

In [ ]:
df = df[df['시설도로명상세주소'].str.contains('(동)')]
# 인덱스 번호 초기화
df = df.reset_index()

df.head(5)

<ipython-input-88-b48ddbe5883a>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[df['시설도로명상세주소'].str.contains('(동)')]


,level_0,index,Unnamed: 0,행정구역명,학원명,분야명,교습계열명,교습과정목록명,,시설도로명주소,시설도로명상세주소,시설도로명우편번호,도로명주소
0,0,1,1,남구,구선생수학교습소,입시.검정 및 보습,보통교과,NaN,보습,대구광역시 남구 현충서길 49,1층일부 (대명동),705824,대구광역시 남구 현충서길 49 1층일부 (대명동)
1,1,2,2,동구,스카이영수학원,입시.검정 및 보습,보통교과,NaN,보습,울산광역시 동구 문재5길 21,", 2층 (방어동)",44094,"울산광역시 동구 문재5길 21, 2층 (방어동)"
2,2,3,3,성남시,어바웃피아노(about piano)학원,기예(대),기예(중),"실용음악(성악),",실용음악(성악),경기도 성남시 수정구 고등로 81,", 503호 (고등동)",13105,"경기도 성남시 수정구 고등로 81, 503호 (고등동)"
3,3,5,5,춘천시,스위첸피아노음악교습소,예능(대),예능(중),피아노(초1),음악,강원 춘천시 동면 만천로 69,"(동면,KCC스위첸아파트)",200852,"강원 춘천시 동면 만천로 69(동면,KCC스위첸아파트)"
4,4,6,6,춘천시,에듀왕영재학원,입시.검정 및 보습,보통교과,"보습,",보습,강원 춘천시 윗무린개길 26,(퇴계동),200943,강원 춘천시 윗무린개길 26(퇴계동)


In [ ]:
# null값 제거
df = df.dropna(subset=['시설도로명주소'])
df = df.dropna(subset=['시설도로명상세주소'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107839 entries, 0 to 107838
Data columns (total 12 columns):
 #   Column                                                                                                                                                                                                                     Non-Null Count   Dtype 
---  ------                                                                                                                                                                                                                     --------------   ----- 
 0   index                                                                                                                                                                                                                      107839 non-null  int64 
 1   Unnamed: 0                                                                                                                                  

# 시설도로명주소 정규표현식 생성

In [ ]:
data_도로명주소 = df['시설도로명주소']
data_도로명주소 = pd.DataFrame(data_도로명주소)
data_도로명주소.head()

In [ ]:
def apply_regex(text):
    # '시' 뒤에 '구'가 오는 경우 또는 '시' 또는 '구' 뒤의 문자열을 제거하는 정규 표현식
    pattern = r'(.*?시\s*[^구]*구)|.*?시'
    match = re.match(pattern, text)

    # 정규 표현식과 일치하는 경우
    if match:
        return match.group()
    # 일치하지 않는 경우 None 반환
    else:
        return None

# '시설도로명주소' 컬럼에 정규 표현식 적용
modified_addresses = data_도로명주소['시설도로명주소'].apply(apply_regex)

modified_addresses = pd.DataFrame(modified_addresses)
modified_addresses.head(5)

,시설도로명주소
0,대구광역시 남구
1,울산광역시 동구
2,경기도 성남시 수정구
3,강원 춘천시
4,강원 춘천시


In [ ]:
modified_addresses.shape

(107839, 1)

# 시설도로명상세주소 정규표현식 생성

In [ ]:
df_시설도로명상세주소 = df['시설도로명상세주소']

df_시설도로명상세주소.head()

0        1층일부 (대명동)
1        , 2층 (방어동)
2      , 503호 (고등동)
3    (동면,KCC스위첸아파트)
4             (퇴계동)
Name: 시설도로명상세주소, dtype: object

In [ ]:
pattern = r'\(.*?\)'

# 추출된 데이터를 저장할 리스트
extracted_data = []

for item in df_시설도로명상세주소:
    # 정규 표현식을 이용하여 괄호와 그 안의 내용을 찾기
    match = re.search(pattern, item)

    # 찾은 데이터가 있다면 추출된 데이터 리스트에 추가
    if match:
        extracted_data.append(match.group())
    else:
        # 패턴과 일치하지 않는 경우 None 추가
        extracted_data.append(None)

extracted_data = pd.DataFrame(extracted_data)
extracted_data.head()

                0
0           (대명동)
1           (방어동)
2           (고등동)
3  (동면,KCC스위첸아파트)
4           (퇴계동)


# 정규식 처리된 데이터 shape확인하기

In [ ]:
modified_addresses.shape, extracted_data.shape

((107839, 1), (107839, 1))

# 데이터 합치기

In [ ]:
data_시_동 = modified_addresses['시설도로명주소'] + extracted_data[0]

# 데이터프레임으로 형식 변환
data_시_동 = pd.DataFrame(data_시_동)

data_시_동.rename(columns={0:'시_동'}, inplace=True)
data_시_동

# 예외 처리 사항확인

In [ ]:
data_시_동[data_시_동['시_동'].str.contains('경기도')]

#'경기도 고양시 덕양구(성사동, 남지빌딩)'은 '경기도 고양시 덕양구(성사동)'으로 변경

,시_동,modified_시_동
2,경기도 성남시 수정구(고등동),경기도 성남시 수정구(고등동)
2234,"경기도 고양시 덕양구(성사동, 남지빌딩)",경기도 고양시 덕양구(성사동)
2235,"경기도 고양시 덕양구(성사동, 남지빌딩)",경기도 고양시 덕양구(성사동)
2236,경기도 고양시 덕양구(삼송동),경기도 고양시 덕양구(삼송동)
2237,경기도 고양시 덕양구(삼송동),경기도 고양시 덕양구(삼송동)
...,...,...
32331,경기도 화성시(기안동),경기도 화성시(기안동)
32332,"경기도 화성시(기안동, 네잎클로버음악학원)",경기도 화성시(기안동)
32333,경기도 화성시(기안동),경기도 화성시(기안동)
32334,"경기도 화성시(안녕동, 아름마을 신한미지엔아파트)",경기도 화성시(안녕동)


In [ ]:
def apply_regex(text):
    # '시' 또는 '구'까지, 그리고 괄호 안에서 '동' 뒤의 ',' 이후의 문자들을 제거하는 정규 표현식
    pattern = r'(.*?[시구])\s*\(([^,]*동).*?\)'
    match = re.match(pattern, text)

    if match:
        return f"{match.group(1)}({match.group(2)})"
    else:
        return None

# '시_동' 컬럼에 정규 표현식 적용
data_시_동['modified_시_동'] = data_시_동['시_동'].apply(apply_regex)

print(data_시_동)

                              시_동      modified_시_동
0                   대구광역시 남구(대명동)     대구광역시 남구(대명동)
1                   울산광역시 동구(방어동)     울산광역시 동구(방어동)
2                경기도 성남시 수정구(고등동)  경기도 성남시 수정구(고등동)
3            강원 춘천시(동면,KCC스위첸아파트)         강원 춘천시(동)
4                     강원 춘천시(퇴계동)       강원 춘천시(퇴계동)
...                           ...               ...
107834              충청북도 충주시(호암동)     충청북도 충주시(호암동)
107835  충청북도 충주시(호암동, 호암시그니처 603)     충청북도 충주시(호암동)
107836              충청북도 충주시(호암동)     충청북도 충주시(호암동)
107837              충청북도 충주시(호암동)     충청북도 충주시(호암동)
107838              충청북도 충주시(호암동)     충청북도 충주시(호암동)

[107839 rows x 2 columns]


In [ ]:
data_시_동

,시_동,modified_시_동
0,대구광역시(대명동),대구광역시(대명동)
1,울산광역시(방어동),울산광역시(방어동)
2,경기도 성남시(고등동),경기도 성남시(고등동)
3,"강원 춘천시(동면,KCC스위첸아파트)",강원 춘천시(동)
4,강원 춘천시(퇴계동),강원 춘천시(퇴계동)
...,...,...
107834,충청북도 충주시(호암동),충청북도 충주시(호암동)
107835,"충청북도 충주시(호암동, 호암시그니처 603)",충청북도 충주시(호암동)
107836,충청북도 충주시(호암동),충청북도 충주시(호암동)
107837,충청북도 충주시(호암동),충청북도 충주시(호암동)


In [ ]:
data_시_동['modified_시_동'].value_counts().head(30)
# 구단위로 나누기
# 상위권 순위중 근접한 동 묶

서울특별시 강남구(대치동)       1419
서울특별시 양천구(목동)         976
서울특별시 양천구(신정동)        807
대구광역시 수성구(범어동)        735
인천광역시 연수구(송도동)        733
부산광역시 해운대구(좌동)        511
서울특별시 노원구(중계동)        484
충청남도 천안시 서북구(불당동)     477
경기도 남양주시(다산동)         455
경기도 부천시(중동)           448
경기도 안양시 동안구(호계동)      444
부산광역시 강서구(명지동)        442
경기도 화성시(반송동)          441
경기도 고양시 일산서구(일산동)     438
경기도 성남시 분당구(정자동)      419
경기도 안양시 동안구(평촌동)      412
경기도 성남시 분당구(수내동)      412
경기도 부천시(상동)           403
서울특별시 강남구(신사동)        382
인천광역시 서구(청라동)         377
전라북도 전주시 덕진구(송천동)     376
서울특별시 서초구(반포동)        370
서울특별시 서초구(방배동)        369
경기도 수원시 영통구(영통동)      365
대전광역시 서구(둔산동)         359
전라북도 전주시 완산구(효자동)     352
서울특별시 노원구(상계동)        350
서울특별시 서초구(서초동)        347
대구광역시 수성구(만촌동)        345
광주광역시 남구(봉선동)         344
Name: modified_시_동, dtype: int64

In [ ]:
data_시_동_top30 = data_시_동['modified_시_동'].value_counts().head(30)

In [ ]:
data_시_동_top30

서울특별시 강남구(대치동)       1419
서울특별시 양천구(목동)         976
서울특별시 양천구(신정동)        807
대구광역시 수성구(범어동)        735
인천광역시 연수구(송도동)        733
부산광역시 해운대구(좌동)        511
서울특별시 노원구(중계동)        484
충청남도 천안시 서북구(불당동)     477
경기도 남양주시(다산동)         455
경기도 부천시(중동)           448
경기도 안양시 동안구(호계동)      444
부산광역시 강서구(명지동)        442
경기도 화성시(반송동)          441
경기도 고양시 일산서구(일산동)     438
경기도 성남시 분당구(정자동)      419
경기도 안양시 동안구(평촌동)      412
경기도 성남시 분당구(수내동)      412
경기도 부천시(상동)           403
서울특별시 강남구(신사동)        382
인천광역시 서구(청라동)         377
전라북도 전주시 덕진구(송천동)     376
서울특별시 서초구(반포동)        370
서울특별시 서초구(방배동)        369
경기도 수원시 영통구(영통동)      365
대전광역시 서구(둔산동)         359
전라북도 전주시 완산구(효자동)     352
서울특별시 노원구(상계동)        350
서울특별시 서초구(서초동)        347
대구광역시 수성구(만촌동)        345
광주광역시 남구(봉선동)         344
Name: modified_시_동, dtype: int64

In [ ]:
data_시_동_top30 = pd.DataFrame(data_시_동_top30)

In [ ]:
data_시_동_top30 = data_시_동_top30.reset_index()
data_시_동_top30

,level_0,index,modified_시_동
0,0,서울특별시 강남구(대치동),1419
1,1,서울특별시 양천구(목동),976
2,2,서울특별시 양천구(신정동),807
3,3,대구광역시 수성구(범어동),735
4,4,인천광역시 연수구(송도동),733
5,5,부산광역시 해운대구(좌동),511
6,6,서울특별시 노원구(중계동),484
7,7,충청남도 천안시 서북구(불당동),477
8,8,경기도 남양주시(다산동),455
9,9,경기도 부천시(중동),448


In [ ]:
data_시_동_top30['index'].sort_values()

13    경기도 고양시 일산서구(일산동)
8         경기도 남양주시(다산동)
17          경기도 부천시(상동)
9           경기도 부천시(중동)
16     경기도 성남시 분당구(수내동)
14     경기도 성남시 분당구(정자동)
23     경기도 수원시 영통구(영통동)
15     경기도 안양시 동안구(평촌동)
10     경기도 안양시 동안구(호계동)
12         경기도 화성시(반송동)
29        광주광역시 남구(봉선동)
28       대구광역시 수성구(만촌동)
3        대구광역시 수성구(범어동)
24        대전광역시 서구(둔산동)
11       부산광역시 강서구(명지동)
5        부산광역시 해운대구(좌동)
0        서울특별시 강남구(대치동)
18       서울특별시 강남구(신사동)
26       서울특별시 노원구(상계동)
6        서울특별시 노원구(중계동)
21       서울특별시 서초구(반포동)
22       서울특별시 서초구(방배동)
27       서울특별시 서초구(서초동)
1         서울특별시 양천구(목동)
2        서울특별시 양천구(신정동)
19        인천광역시 서구(청라동)
4        인천광역시 연수구(송도동)
20    전라북도 전주시 덕진구(송천동)
25    전라북도 전주시 완산구(효자동)
7     충청남도 천안시 서북구(불당동)
Name: index, dtype: object

In [ ]:
''' 묶을 수 있을것 같은

          경기도 부천시(상동)
          경기도 부천시(중동)

     경기도 성남시 분당구(수내동)
     경기도 성남시 분당구(정자동)

     경기도 안양시 동안구(평촌동)
     경기도 안양시 동안구(호계동)

       대구광역시 수성구(만촌동)
       대구광역시 수성구(범어동)

        서울특별시 강남구(대치동)

       서울특별시 노원구(상계동)
       서울특별시 노원구(중계동)

       서울특별시 서초구(반포동)
       서울특별시 강남구(신사동)

       서울특별시 서초구(방배동)
       서울특별시 서초구(서초동)

        서울특별시 양천구(목동)
        서울특별시 양천구(신정동)
'''